# Chapter 3: P2SH Script Engineering> **Reference**: `book/manuscript/Chapter 3 P2SH Script Engineering - From Multi-signature to Time Locks.md`  > **Code Examples**: `code/chapter03/`  > **Last Updated**: 2025-12-06---Bitcoin's first real programmability emerges with Pay-to-Script-Hash (P2SH), where complex spending conditions are elegantly hidden behind a simple 20-byte hash. This chapter bridges the gap between basic P2PKH transactions and Taproot's sophisticated script trees, demonstrating how Bitcoin's script system enables real-world applications like corporate treasury management and time-locked inheritance planning.

P2SH enables any script to be represented by a compact 20-byte hash, moving script complexity from the UTXO set to spending time.### The Two-Stage Verification ModelP2SH operates through two distinct phases:**Stage 1: Hash Verification**```OP_HASH160 <script_hash> OP_EQUAL```**Stage 2: Script Execution**```<revealed_script> → Execute as Bitcoin Script```### P2SH Address Generation ProcessP2SH follows the same Hash160 → Base58Check pattern covered in Chapter 1, but hashes the script instead of a public key:```Script Serialization → hex_encoded_scriptHash160(script)     → 20_bytes_script_hash  Version + Base58Check → 3...address (mainnet)```All P2SH addresses begin with "3" on mainnet and "2" on testnet, immediately distinguishing them from P2PKH addresses.### ScriptSig Construction PatternThe unlocking script (ScriptSig) for P2SH follows a specific pattern:```<script_data> <serialized_redeem_script>```Where `<script_data>` contains the values needed to satisfy the redeem script's conditions, and `<serialized_redeem_script>` is the original script whose hash matches the locking script.

Enterprise Bitcoin custody typically requires multi-party authorization to prevent single points of failure. A 2-of-3 multisignature scheme ensures that no single person can unilaterally access funds while maintaining operational flexibility.### Business Scenario: Startup Treasury ManagementConsider a blockchain startup with three key stakeholders:- **Alice**: CEO with operational authority- **Bob**: CTO with technical oversight  - **Carol**: CFO with financial controlsTheir treasury policy requires any two signatures for fund movement, preventing both single-person risk and requiring unanimous consensus.### Multi-signature Script ConstructionThe redeem script implements the 2-of-3 logic using Bitcoin's `OP_CHECKMULTISIG` opcode:### bitcoinutils Function Analysis**`Script([...])` Constructor**: Creates a Script object from a list of opcodes and data. The library automatically handles the encoding of opcodes like `'OP_2'` into their byte representations (`0x52`).**`P2shAddress.from_script(script)`**: Generates a P2SH address by:1. Serializing the script to bytes2. Computing Hash160(script) 3. Adding version byte (0x05 for mainnet, 0xc4 for testnet)4. Applying Base58Check encoding**Script Serialization**: The redeem script serializes to:`522102898711e6bf63f5cbe1b38c05e89d6c391c59e9f8f695da44bf3d20ca674c8519210284b5951609b76619a1ce7f48977b4312ebe226987166ef044bfb374ceef63af5210317aa89b43f46a0c0cdbd9a302f2508337ba6a06d123854481b52de9c2099601153ae`Breaking this down:- `52`: OP_2- `21`: Push 33 bytes (compressed public key)- `02898711...`: Alice's public key- `21`: Push 33 bytes  - `0284b595...`: Bob's public key- `21`: Push 33 bytes- `0317aa89...`: Carol's public key- `53`: OP_3- `ae`: OP_CHECKMULTISIG### Spending the Multi-signature UTXOWhen Alice and Bob decide to authorize a payment, they must provide their signatures in the correct order along with the redeem script:### bitcoinutils Signature Functions**`private_key.sign_input(tx, input_index, script)`**: Creates an ECDSA signature for a specific transaction input using the provided script for the signature hash calculation. The script parameter should be the redeem script for P2SH inputs.**`script.to_hex()`**: Serializes the Script object into its hexadecimal byte representation, which is pushed as data onto the stack during script execution.### Multi-signature Stack Execution AnalysisLet's trace through the complete script execution using our real transaction data, understanding Bitcoin Core's two-phase P2SH execution mechanism:**Transaction ID**: `e68bef534c7536300c3ae5ccd0f79e031cab29d262380a37269151e8ba0fd4e0`

**Initial State:**```│ (empty)                                │└────────────────────────────────────────┘```### 1. PUSH OP_0: Multisig bug workaroundBitcoin's OP_CHECKMULTISIG has a known off-by-one bug that pops an extra item from the stack. An OP_0 is pushed to compensate.```│ 00 (op_zero)                           │└────────────────────────────────────────┘```### 2. PUSH Alice's Signature: First authorization```│ 30440220694f...7a6501 (alice_sig)      ││ 00 (op_zero)                           │└────────────────────────────────────────┘```### 3. PUSH Bob's Signature: Second authorization  ```│ 3044022065f8...fd9e01 (bob_sig)        ││ 30440220694f...7a6501 (alice_sig)      ││ 00 (op_zero)                           │└────────────────────────────────────────┘```### 4. PUSH Redeem Script: Reveal the spending conditions```│ 522102898711...601153ae (redeem_script)  ││ 3044022065f8...fd9e01 (bob_sig)          ││ 30440220694f...7a6501 (alice_sig)        ││ 00 (op_zero)                             │└──────────────────────────────────────────┘```### 5. OP_HASH160: Verify script hash matchesThe P2SH locking script `OP_HASH160 <script_hash> OP_EQUAL` is executed:```│ dd81b5beb3d8...5cb0ca (computed_hash)    ││ 3044022065f8...fd9e01 (bob_sig)          ││ 30440220694f...7a6501 (alice_sig)        ││ 00 (op_zero)                             │└──────────────────────────────────────────┘```### 6. PUSH Expected Hash: From locking script```│ dd81b5beb3d8...5cb0ca (expected_hash)    ││ dd81b5beb3d8...5cb0ca (computed_hash)    ││ 3044022065f8...fd9e01 (bob_sig)          ││ 30440220694f...7a6501 (alice_sig)        ││ 00 (op_zero)                             │└──────────────────────────────────────────┘```### 7. OP_EQUAL: Confirm hash match```│ 1 (true)                               ││ 3044022065f8...fd9e01 (bob_sig)        ││ 30440220694f...7a6501 (alice_sig)      ││ 00 (op_zero)                           │└────────────────────────────────────────┘```**(Phase 1 complete: Hash verification successful)**

**Critical Point**: Bitcoin Core recognizes the P2SH pattern and transitions to a second validation phase by:1. **Detects P2SH pattern**: `OP_HASH160 <hash> OP_EQUAL` 2. **Resets stack**: Back to post-scriptSig state (discards TRUE)3. **Extracts redeem script**: From original scriptSig data4. **Prepares clean execution**: For redeem script with signature data**Stack Reset to Post-ScriptSig State:**```│ 3044022065f8...fd9e01 (bob_sig)        ││ 30440220694f...7a6501 (alice_sig)      ││ 00 (op_zero)                           │└────────────────────────────────────────┘```**(TRUE is discarded - redeem script begins with clean stack)**

Bitcoin Core now executes the redeem script: `OP_2 alice_pk bob_pk carol_pk OP_3 OP_CHECKMULTISIG`### 8. OP_2: Push required signature count```│ 2 (required_sigs)                      ││ 3044022065f8...fd9e01 (bob_sig)        ││ 30440220694f...7a6501 (alice_sig)      ││ 00 (op_zero)                           │└────────────────────────────────────────┘```### 9-11. PUSH Public Keys: Load verification keys```│ 0317aa89b43f...996011 (carol_pk)       ││ 0284b5951609...eef63af5 (bob_pk)       ││ 02898711e6bf...674c8519 (alice_pk)     ││ 2 (required_sigs)                      ││ 3044022065f8...fd9e01 (bob_sig)        ││ 30440220694f...7a6501 (alice_sig)      ││ 00 (op_zero)                           │└────────────────────────────────────────┘```### 12. OP_3: Push total key count```│ 3 (total_keys)                         ││ 0317aa89b43f...996011 (carol_pk)       ││ 0284b5951609...eef63af5 (bob_pk)       ││ 02898711e6bf...674c8519 (alice_pk)     ││ 2 (required_sigs)                      ││ 3044022065f8...fd9e01 (bob_sig)        ││ 30440220694f...7a6501 (alice_sig)      ││ 00 (op_zero)                           │└────────────────────────────────────────┘```### 13. OP_CHECKMULTISIG: Verify signaturesThe opcode consumes:- Key count (3)- Public keys (Alice, Bob, Carol)  - Signature count (2)- Signatures (Alice's, Bob's)- Extra item (OP_0, due to bug)Verification process:1. Alice's signature verified against Alice's public key ✓2. Bob's signature verified against Bob's public key ✓  3. Required threshold (2-of-3) satisfied ✓### Final State: Multisig Verification Complete```│ 1 (true)                               │└────────────────────────────────────────┘```**(P2SH execution successful: Clean two-phase verification)**

CheckSequenceVerify (CSV) enables relative time locks, where spending is delayed relative to when the UTXO was created. Let's examine a real implementation using actual testnet data.### Real-World Implementation: 3-Block Time Lock**Transaction ID**: `34f5bf0cf328d77059b5674e71442ded8cdcfc723d0136733e0dbf180861906f`This transaction demonstrates a P2SH script that combines CSV time lock with P2PKH signature verification—a common pattern for inheritance and escrow applications.### CSV Script ConstructionUnlike complex conditional branching, the time lock uses a simple linear script:### bitcoinutils CSV Functions**`Sequence(TYPE_RELATIVE_TIMELOCK, blocks)`**: Creates a sequence object for relative block-based delays. The sequence value encodes the time constraint that will be enforced by OP_CHECKSEQUENCEVERIFY.**`seq.for_script()`**: Returns the sequence value formatted for use in script opcodes (pushes the delay value onto the stack).**`seq.for_input_sequence()`**: Returns the sequence value for the transaction input's sequence field, which CSV validates against.### Spending the Time-Locked UTXO### CSV Stack Execution AnalysisLet's trace through the execution using real transaction data from our testnet example:**ScriptSig Data**:- Signature: `30440220...` (71 bytes)- Public Key: `0250be5fc44ec580c387bf45df275aaa8b27e2d7716af31f10eeed357d126bb4d3` (33 bytes)  - Redeem Script: `53b27576a9145cdc...88ac` (28 bytes)

**(Stack reset mechanism applies - see multisig section for details)**

**Initial State** (after P2SH reset):```│ 0250be5fc44ec...4d3 (pubkey)           ││ 30440220a1b2...c3d401 (signature)      │└────────────────────────────────────────┘```### 1. PUSH 3: Time delay requirement```│ 3 (delay_blocks)                       ││ 0250be5fc44ec...4d3 (pubkey)           ││ 30440220a1b2...c3d401 (signature)      │└────────────────────────────────────────┘```### 2. OP_CHECKSEQUENCEVERIFY: Verify time lockCSV validates that the transaction input's sequence number ≥ 3:```│ 3 (delay_blocks)                       ││ 0250be5fc44ec...4d3 (pubkey)           ││ 30440220a1b2...c3d401 (signature)      │└────────────────────────────────────────┘```**(Verification: nSequence ≥ 3 blocks since UTXO creation)**### 3. OP_DROP: Remove delay value```│ 0250be5fc44ec...4d3 (pubkey)           ││ 30440220a1b2...c3d401 (signature)      │└────────────────────────────────────────┘```### 4. OP_DUP: Begin P2PKH verification```│ 0250be5fc44ec...4d3 (pubkey)           ││ 0250be5fc44ec...4d3 (pubkey)           ││ 30440220a1b2...c3d401 (signature)      │└────────────────────────────────────────┘```### 5. OP_HASH160: Hash public key```│ 5cdc28d6b1876...cabaadcc (pubkey_hash) ││ 0250be5fc44ec...4d3 (pubkey)           ││ 30440220a1b2...c3d401 (signature)      │└────────────────────────────────────────┘```### 6. PUSH Expected Hash: From redeem script```│ 5cdc28d6b1876...cabaadcc (expected_hash) ││ 5cdc28d6b1876...cabaadcc (computed_hash) ││ 0250be5fc44ec...4d3 (pubkey)             ││ 30440220a1b2...c3d401 (signature)        │└──────────────────────────────────────────┘```### 7. OP_EQUALVERIFY: Confirm hash match```│ 0250be5fc44ec...4d3 (pubkey)           ││ 30440220a1b2...c3d401 (signature)      │└────────────────────────────────────────┘```### 8. OP_CHECKSIG: Final signature verification```│ 1 (true)                               │└────────────────────────────────────────┘```**(Time lock satisfied and signature verified - CSV spending successful)**### Time Lock Error Handling**Common Error: `non-BIP68-final`**If you attempt to spend before the time lock expires:The transaction is rejected because `nSequence < required_delay`, violating the CSV constraint.### CSV Applications**Digital Inheritance**: Assets automatically become accessible to heirs after a specified period of owner inactivity.**Business Continuity**: Corporate funds can include automatic release mechanisms for operational emergencies.**Payment Channels**: Lightning Network uses CSV to enforce settlement delays, enabling dispute resolution periods.

P2SH extends Bitcoin Script from simple single-signature authorization to complex multi-party and temporal conditions, while maintaining the same compact address format.### P2SH's Inherent LimitationsHowever, while P2SH improves efficiency by replacing explicit scripts with compact hashes, it still exposes the full redeem script during spending. This means every possible condition—regardless of which branch is taken—must be revealed when funds are spent. There's no way to selectively reveal only the relevant branch.This design makes P2SH inherently linear and opaque in structure. Unlike Taproot, it cannot express script trees or use Merkle branches to hide unused logic. Every signature path, time lock clause, or fallback condition is fully exposed on-chain.In addition, since redeem scripts must be included in the scriptSig, P2SH transactions carry significant overhead in input size. This leads to higher fees and lower scalability, especially for multisig or inheritance setups.Taproot addresses these limitations directly by allowing complex scripts to remain entirely hidden until needed, and by embedding them in a tree structure where only the executed path is revealed.

This chapter bridged the gap between basic P2PKH transactions and Taproot's advanced capabilities by exploring P2SH's two fundamental patterns: multi-signature authorization and time-locked conditions.**Key Concepts Mastered:****Two-Stage Verification**: P2SH's hash-then-execute model provides the conceptual foundation for Taproot's commitment schemes, where complex scripts remain private until spending.**Multi-party Authorization**: The 2-of-3 multisignature pattern demonstrates how Bitcoin Script handles conditional logic and multiple verification requirements—skills essential for understanding Taproot's script tree execution.**Temporal Constraints**: CSV-based time locks introduce relative time concepts that underpin Lightning Network and other Layer 2 protocols built on Taproot foundations.**Stack-Based Programming**: Detailed stack execution traces for both multisig and time lock scenarios provide the analytical skills needed to debug and optimize Taproot script paths.**bitcoinutils Proficiency**: Practical experience with Script construction, P2shAddress generation, and signature creation prepares developers for Taproot's more sophisticated primitives.**Real Transaction Analysis**: Working with actual testnet transactions and mempool data builds the empirical skills needed for production Taproot development.The progression from P2PKH's simple signature verification through P2SH's complex conditional logic establishes the foundation for Taproot's revolutionary approach: making sophisticated smart contracts indistinguishable from simple payments while providing unprecedented script flexibility and privacy.In the next chapter, we'll examine how SegWit's witness structure revolutionized transaction malleability and fee calculation—concepts that directly enable Taproot's efficiency improvements and form the basis for understanding P2TR's witness-based spending paths.